# Clasificador de Comentarios usando Modelos entrenados RoBERTuito

Autores:
- Pablo Quito
- Juan Valdiviezo 

In [ ]:
%pip install transformers
%pip install language-tool-python
%pip install autocorrect
%pip install python-dotenv

In [ ]:
import language_tool_python
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup, AutoTokenizer
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap
from autocorrect import Speller
from transformers import RobertaModel, AutoModel
import torch.nn as nn
from google.colab import drive
import os
from dotenv import load_dotenv

In [ ]:
BASE_PATH = '/content/drive/MyDrive/Intelektubies/'
DATA_PATH = os.path.join(BASE_PATH, 'Datos/Raw Comments')
MODELS_PATH = os.path.join(BASE_PATH, 'Modelos/RoBERTuito_folds/v5')
NOTEBOOKS_PATH = os.path.join(BASE_PATH, 'Cuadernos Jupyter')
COMMENTARIES_PATH = os.path.join(BASE_PATH, 'Datos/Comentarios clasificados')

In [ ]:
drive.mount('/content/drive')

In [ ]:
#CONEXIÓN CON SUPABASE
# Carga las variables de entorno desde el archivo .env
PATH_ENV = " "
load_dotenv(PATH_ENV)

# Obtiene las variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")
DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

In [ ]:
!apt-get install openjdk-17-jdk -y

In [ ]:
# Inicializar
RANDOM_SEED = 42
MAX_LEN = 130 #antes 200
BATCH_SIZE = 16 #antes 32
NCLASSES = 4
tool = language_tool_python.LanguageTool('es')

In [ ]:
#Tokenizacion
# RoBERTuito: 'pysentimiento/robertuito-base-uncased-emotion'

PRE_TRAINED_MODEL_NAME = 'pysentimiento/robertuito-base-uncased-emotion'

tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)  # Usa AutoTokenizer para elegir el tokenizador correcto

In [ ]:
# Use the GPU power >:D
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Preprocesado


*   Cargar tabla de comentarios de la Base de datos



In [ ]:
#Estructura de la base de datos
from typing import Optional, List

from sqlalchemy import (
    String,
    Integer,
    Float,
    Text,
    ForeignKey,
    CheckConstraint
)
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship


class Base(DeclarativeBase):
    pass


class Facultad(Base):
    __tablename__ = "facultad"

    facultad_id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    nombre: Mapped[str] = mapped_column(String(100), nullable=False)

    carreras: Mapped[List["Carrera"]] = relationship("Carrera", back_populates="facultad")

    def __repr__(self) -> str:
        return f"<Facultad(facultad_id={self.facultad_id}, nombre={self.nombre})>"


class Carrera(Base):
    __tablename__ = "carrera"

    carrera_id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    facultad_id: Mapped[int] = mapped_column(ForeignKey("facultad.facultad_id"), nullable=False)
    nombre: Mapped[str] = mapped_column(String(100), nullable=False)

    facultad: Mapped["Facultad"] = relationship("Facultad", back_populates="carreras")
    carrera_asignaturas: Mapped[List["CarreraAsignatura"]] = relationship("CarreraAsignatura", back_populates="carrera")

    def __repr__(self) -> str:
        return f"<Carrera(carrera_id={self.carrera_id}, nombre={self.nombre})>"


class Asignatura(Base):
    __tablename__ = "asignatura"

    asignatura_id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    nombre: Mapped[str] = mapped_column(String(100), nullable=False)
    codigo: Mapped[Optional[int]] = mapped_column(Integer, nullable=True)
    vector_competencias: Mapped[Optional[str]] = mapped_column(Text, nullable=True)

    carrera_asignaturas: Mapped[List["CarreraAsignatura"]] = relationship("CarreraAsignatura", back_populates="asignatura")
    evaluaciones: Mapped[List["Evaluacion"]] = relationship("Evaluacion", back_populates="asignatura")

    def __repr__(self) -> str:
        return f"<Asignatura(asignatura_id={self.asignatura_id}, nombre={self.nombre})>"


class CarreraAsignatura(Base):
    __tablename__ = "carrera_asignatura"

    carrera_id: Mapped[int] = mapped_column(ForeignKey("carrera.carrera_id"), primary_key=True)
    asignatura_id: Mapped[int] = mapped_column(ForeignKey("asignatura.asignatura_id"), primary_key=True)

    carrera: Mapped["Carrera"] = relationship("Carrera", back_populates="carrera_asignaturas")
    asignatura: Mapped["Asignatura"] = relationship("Asignatura", back_populates="carrera_asignaturas")

    def __repr__(self) -> str:
        return f"<CarreraAsignatura(carrera_id={self.carrera_id}, asignatura_id={self.asignatura_id})>"


class Docente(Base):
    __tablename__ = "docente"

    docente_id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    cedula: Mapped[Optional[str]] = mapped_column(String(100), nullable=True)
    nombre: Mapped[str] = mapped_column(String(100), nullable=False)
    vector_competencias: Mapped[Optional[str]] = mapped_column(Text, nullable=True)

    evaluaciones: Mapped[List["Evaluacion"]] = relationship("Evaluacion", back_populates="docente")

    def __repr__(self) -> str:
        return f"<Docente(docente_id={self.docente_id}, nombre={self.nombre})>"


class Evaluacion(Base):
    __tablename__ = "evaluacion"

    evaluacion_id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    docente_id: Mapped[int] = mapped_column(ForeignKey("docente.docente_id"), nullable=False)
    asignatura_id: Mapped[int] = mapped_column(ForeignKey("asignatura.asignatura_id"), nullable=False)
    periodo: Mapped[int] = mapped_column(Integer, nullable=False)
    nota_comentarios: Mapped[float] = mapped_column(Float, nullable=True)
    nota_evaluacion: Mapped[float] = mapped_column(Float, nullable=False)

    docente: Mapped["Docente"] = relationship("Docente", back_populates="evaluaciones")
    asignatura: Mapped["Asignatura"] = relationship("Asignatura", back_populates="evaluaciones")
    comentarios: Mapped[List["ComentarioEvaluacion"]] = relationship("ComentarioEvaluacion", back_populates="evaluacion")

    def __repr__(self) -> str:
        return (f"<Evaluacion(evaluacion_id={self.evaluacion_id}, docente_id={self.docente_id}, "
                f"asignatura_id={self.asignatura_id}, periodo={self.periodo}, "
                f"nota_comentarios={self.nota_comentarios}, nota_evaluacion={self.nota_evaluacion})>")


class ComentarioEvaluacion(Base):
    __tablename__ = "comentario_evaluacion"
    __table_args__ = (
        CheckConstraint(
            "etiqueta in ('Positiva','Negativa','Neutro','Alerta')", name="chk_etiqueta"
        ),
    )

    comentario_id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    evaluacion_id: Mapped[int] = mapped_column(ForeignKey("evaluacion.evaluacion_id"), nullable=False)
    comentario: Mapped[str] = mapped_column(Text, nullable=False)
    etiqueta: Mapped[str] = mapped_column(String(10), nullable=False)

    evaluacion: Mapped["Evaluacion"] = relationship("Evaluacion", back_populates="comentarios")

    def __repr__(self) -> str:
        return (f"<ComentarioEvaluacion(comentario_id={self.comentario_id}, "
                f"evaluacion_id={self.evaluacion_id}, etiqueta={self.etiqueta})>")


In [ ]:
#Crear motor y la session
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine(DATABASE_URL)

# Crea una sesión
Session = sessionmaker(bind=engine)
session = Session()

### Aplicacion del modelo

In [ ]:
#MODELO
class RoBERTtuitoSentimentClassifier(nn.Module):
    def __init__(self,n_classes):
        super(RoBERTtuitoSentimentClassifier,self).__init__()
        #self.roberta = RobertaModel.from_pretrained(PRE_TRAINED_MODEL)
        self.roberta = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME,add_pooling_layer=False)
        self.drop = nn.Dropout(p=0.3)
        self.linear = nn.Linear(self.roberta.config.hidden_size,n_classes)
    def forward(self, input_ids, attention_mask):
        output = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # RoBERTa doesn't use pooler_output like BERT
        # Use the first token's hidden state from the last_hidden_state
        cls_output = output['last_hidden_state'][:, 0, :]  # [batch_size, hidden_size]

        drop_output = self.drop(cls_output)
        output = self.linear(drop_output)
        return output

In [ ]:
def load_model(model_idx):
    model_path = os.path.join(MODELS_PATH, f'model_fold_{model_idx}.pth')
    model = RoBERTtuitoSentimentClassifier(NCLASSES)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

In [ ]:
# Clasificación de sentimiento
def classify_sentiment(model, review_text):
    encoding_review = tokenizer.encode_plus(
        review_text, max_length=10, add_special_tokens=True,
        return_token_type_ids=False, padding='max_length',
        return_attention_mask=True, return_tensors='pt'
    )
    input_ids = encoding_review['input_ids'].to(device)
    attention_mask = encoding_review['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask)
        _, prediction = torch.max(output, dim=1)

    return prediction.item()

In [ ]:
def corregir_text(text):
    matches = tool.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text

In [ ]:
BATCH_SIZE = 100

try:
    comentarios = session.query(ComentarioEvaluacion).all()
    corregidos_total = 0

    for i in range(0, len(comentarios), BATCH_SIZE):
        batch = comentarios[i:i + BATCH_SIZE]

        for c in batch:
            original = c.comentario
            corregido = corregir_text(original)

            if original != corregido:
                print(f"[{c.comentario_id}] Comentario corregido:\n  Antes: {original}\n  Después: {corregido}\n")
                c.comentario = corregido
                corregidos_total += 1

        session.commit()

    print(f"Se corrigieron {corregidos_total} comentarios.")

except Exception as e:
    session.rollback()
    print("Error:", e)

finally:
    session.close()


Guardar pre-procesado

*   Necesario para no perder el progreso
*   Se puede iniciar desde este punto si no se hace el preprocesado

In [ ]:
# Cargar los datos desde la base de datos
with Session() as session:
    query = session.query(ComentarioEvaluacion.comentario_id,ComentarioEvaluacion.evaluacion_id, ComentarioEvaluacion.comentario, ComentarioEvaluacion.etiqueta)
    df_comentarios = pd.read_sql(query.statement, session.bind)

df_comentarios.head()

In [ ]:
for i in range(6):
    model = load_model(i)
    print(f"Modelo {i} cargado:")
    df_comentarios[f'f{i}'] = df_comentarios['comentario'].apply(lambda x: classify_sentiment(model, x))
    del model  # Liberar memoria de GPU
    torch.cuda.empty_cache()

# Nueva columna con la moda
df_comentarios['sentimiento'] = df_comentarios[[f'f{i}' for i in range(6)]].mode(axis=1)[0]

# Mapeo de sentimiento predicho (moda)
sentiment_mapping = {0: 'Negativo', 1: 'Neutral', 2: 'Positivo', 3: 'Alerta'}
df_comentarios['sentimiento'] = df_comentarios['sentimiento'].map(sentiment_mapping)

# Eliminar columnas temporales f0,f1,f2,f3,f4,f5
df_comentarios = df_comentarios.drop(columns=[f'f{i}' for i in range(6)])

df_comentarios.head()


In [ ]:
from sqlalchemy import update
with Session() as session:
    for _, row in df_comentarios.iterrows():
        stmt = (
            update(ComentarioEvaluacion)
            .where(ComentarioEvaluacion.comentario_id == row['comentario_id'])
            .values(etiqueta=row['sentimiento'])
        )
        session.execute(stmt)
    session.commit()

### Calificacion

Puntuación Total = (P × +2) + (N × 0) + (Neg × -1) + (A × -5)


Calificación Final = ((Puntuación Total - Puntuación Mínima) / (Puntuación Máxima - Puntuación Mínima)) × 100


In [ ]:
def calc_calificacion(labels):
    # Ponderaciones
    ponderaciones = {
        'Positivo': 2,
        'Neutral': 0,
        'Negativo': -1,
        'Alerta': -5
    }

    # conteo
    conteo = {
        'Positivo': 0,
        'Neutral': 0,
        'Negativo': 0,
        'Alerta': 0
    }

    for label in labels:
        if label in conteo:
            conteo[label] += 1
        else:
            print(f"Etiqueta desconocida: {label}")

    total_comentarios = sum(conteo.values())

    # Fix: evitar division por cero si no hay comentarios
    if total_comentarios == 0:
        return 0


    puntuacion_total = sum(conteo[etiqueta] * ponderaciones[etiqueta] for etiqueta in conteo)

    # Max y min
    puntuacion_maxima = total_comentarios * ponderaciones['Positivo']
    puntuacion_minima = total_comentarios * ponderaciones['Alerta']

    # Normalizar la puntuacion
    calificacion_final = ((puntuacion_total - puntuacion_minima) / (puntuacion_maxima - puntuacion_minima)) * 100

    calificacion_final = max(0, min(100, calificacion_final))

    return calificacion_final


In [ ]:
#Agrupar por sentimiento y calcular calificación
df_calificacion = df_comentarios.groupby('evaluacion_id')['sentimiento'].apply(list).reset_index()
df_calificacion['calificacion'] = df_calificacion['sentimiento'].apply(calc_calificacion)
df_calificacion.head()

In [ ]:
# Actualizar tabla evaluacion
try:
    with Session() as session:
        for _, row in df_calificacion.iterrows():
            stmt = (
                update(Evaluacion)
                .where(Evaluacion.evaluacion_id == row['evaluacion_id'])
                .values(nota_comentarios=row['calificacion'])
            )
            session.execute(stmt)
        session.commit()
        print("Actualización completada correctamente.")

except Exception as e:
    session.rollback()
    print("Ocurrió un error al actualizar la tabla Evaluacion:", e)

In [ ]:
# Agrupar sentimientos y calcular la calificación por docente y carrera
df_calificacion = df_final.groupby(['Identificación Docente', 'Facultad', 'Carrera'])['sentimiento'].apply(list).reset_index()
df_calificacion['calificacion'] = df_calificacion['sentimiento'].apply(calc_calificacion)

# Adjuntar sentimientos y crear columna de alerta
df_aux = df_final.groupby(['Identificación Docente', 'Facultad', 'Carrera'])['sentimiento'].apply(lambda x: ', '.join(x)).reset_index()
df_aux = df_aux.merge(df_calificacion[['Identificación Docente', 'Facultad', 'Carrera', 'calificacion']],
                      on=['Identificación Docente', 'Facultad', 'Carrera'], how='left')


In [ ]:
# Adjuntar el nombre del docente
df_aux = df_aux.merge(df_final[['Identificación Docente', 'Docente']].drop_duplicates(),
                      on='Identificación Docente', how='left')

# Reordenar columnas para mayor claridad
df_aux = df_aux[['Identificación Docente', 'Docente', 'Facultad','Carrera', 'sentimiento', 'calificacion', 'Alerta']]


In [ ]:
#Guardar df_aux
arch_dife = 'nuevo'
output_file = os.path.join(COMMENTARIES_PATH, f'Calificación_Docente_Comentario_{arch_dife}.xlsx')
df_aux.to_excel(output_file, index=False)

In [ ]:
#adjuntar columna de codigo con el numero 375
df_aux['Codigo'] = 375

#Borrar la colimna Docente
df_aux = df_aux.drop(columns=['Docente'])

df_aux